# Evaluate metrics and record fit and score times

#### Authors:

* Juan Carlos Alfaro Jiménez

---

**Note:** All the parameters are provided by `guildai` via environment variables with prefix `FLAG`.

---

In this notebook, we evaluate

## 1. Data

First, we load the data to fit (`X`) and the target rankings to try to predict (`Y`):

---

**Note**: We use the `fetch_openml` function because the benchmark data has been uploaded to the `OpenML` repository.

---

In [ ]:
import os

In [ ]:
data_id = os.environ.get("FLAG_DATA_ID")

In [ ]:
from sklearn.datasets import fetch_openml

In [ ]:
data = fetch_openml(data_id=data_id)

In [ ]:
X, Y = data["data"], data["target"]

Let us print the name of the dataset:

In [ ]:
name = data["details"]["name"]

In [ ]:
f"The experiment will be run using the {name} dataset."

## 2. Estimator

Second, we initialize the estimator object (`estimator`) to use to fit the data:

In [ ]:
module = os.environ.get("GUILD_OP").split(":")[0]

In [ ]:
estimator_type = os.environ.get("FLAG_ESTIMATOR_TYPE")

In [ ]:
estimator = __import__(module).estimator

## 3. Cross-validation strategy

Third, we define the strategy to evaluate the performance of the cross-validated estimator on the test dataset. In particular, we use a $ r \times k $ cross-validation method:

In [ ]:
n_splits = os.environ.get("FLAG_N_SPLITS")

In [ ]:
n_repeats = os.environ.get("FLAG_N_REPEATS")

In [ ]:
random_state = os.environ.get("FLAG_RANDOM_STATE")

In [ ]:
from sklearn.model_selection import RepeatedKFold

In [ ]:
cv = RepeatedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=random_state)